<a href="https://colab.research.google.com/github/liorZucker11/cloud-computing/blob/main/Shablool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [383]:
#!pip install ipywidgets
#!pip install IPython
from IPython.display import display, HTML
import ipywidgets as widgets

In [384]:
# from IPython.display import HTML

# html_code = """
# <html>
#   <head>
#     <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
#     <meta name="viewport" content="width=device-width, initial-scale=1" />
#     <meta http-equiv="X-UA-Compatible" content="IE=edge" />
#     <link href="https://fonts.googleapis.com/css?family=Poppins" rel="stylesheet" />
#     <style>

#       body {
#         font-family: 'Poppins', sans-serif;
#         margin: 0;
#       }

#       .s130 {
#         min-height: 100vh;
#         display: flex;
#         justify-content: center;
#         align-items: center;
#         background-position: bottom right;
#         background-repeat: no-repeat;
#         background-size: 100%;
#         padding: 15px;
#       }

#       form {
#         width: 100%;
#         max-width: 790px;
#         padding-top: 24vh;
#       }

#       .inner-form {
#         display: flex;
#         width: 100%;
#         justify-content: space-between;
#         align-items: center;
#         box-shadow: 0px 8px 20px 0px rgba(0, 0, 0, 0.15);
#         border-radius: 34px;
#         overflow: hidden;
#         margin-bottom: 30px;
#       }

#       .input-field {
#         height: 68px;
#         flex-grow: 1;
#       }

#       input {
#         height: 80%;
#         background: transparent;
#         border: 0;
#         display: block;
#         width: 80%;
#         padding: 10px 20px;
#         font-size: 16px;
#         color: #000;
#       }

#       .btn-search {
#         height: 100%;
#         width: 100%;
#         white-space: nowrap;
#         font-size: 16px;
#         color: #fff;
#         border: 0;
#         cursor: pointer;
#         position: relative;
#         z-index: 0;
#         background: #1877f2;
#         transition: all .2s ease-out, color .2s ease-out;
#         font-weight: 300;
#       }

#       .logo {
#         max-width: 70%;
#         height: 70%;
#         margin-bottom: 10px;
#       }

#       .info {
#         font-size: 15px;
#         color: #ccc;
#         padding-left: 26px;
#       }
#     </style>
#   </head>
#   <body>
#     <div class="s130">
#       <form>
#         <img class="logo" src="https://i.pinimg.com/564x/1c/81/bf/1c81bf9a5861a973a6ad938098a0845f.jpg" alt="Logo">
#         <div class="inner-form">
#           <div class="input-field">
#             <input id="search" type="text" placeholder="What are you looking for?" />
#           </div>
#           <div class="input-field">
#             <button class="btn-search" type="submit">SEARCH</button>
#           </div>
#         </div>
#       </form>
#     </div>
#   </body>
# </html>

# """

# display(HTML(html_code))

In [385]:
from IPython.display import display
import ipywidgets as widgets

container=widgets.Box()
results_output=widgets.Output()

# Function to simulate a search operation and display results
def perform_search(query):
    return [f"Result {i+1} for '{query}'" for i in range(5)]

# Function to update the results display
def update_results_display(search_results):
    results_output.layout.visibility = "visible"
    results_output.clear_output()
    with results_output:
        if search_results:
            for result in search_results:
                print(result)
        else:
            print("No results found.")

# Button actions
def statistics_action(b):
    print("Statistics action triggered")

def create_search_screen():
    def admin_screen_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_admin_screen()

    # Create buttons with actions
    admin_screen_button = widgets.Button(description="Admin Screen", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    statistics_button = widgets.Button(description="Statistics", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    admin_screen_button.on_click(admin_screen_action)
    statistics_button.on_click(statistics_action)

    # Adjust layout for input field, button, and results output
    input_layout = widgets.Layout(width='98%', justify_content='center')
    button_layout = widgets.Layout(width='98%', justify_content='center', margin='10px 0 0 0')
    results_layout = widgets.Layout(border='1px solid black', border_radius='5px', width='95%', margin='20px 0', padding='10px', visibility='hidden')

    # Create the search text box, button, and results output
    search_input = widgets.Text(value='', placeholder='Type here to search', description='', disabled=False, layout=input_layout)
    search_button = widgets.Button(description='SEARCH', button_style='success', tooltip='Click to search', icon='search', layout=button_layout)
    results_output = widgets.Output(layout=results_layout)
    search_button.on_click(search_button)

    # Display logo as a widget
    with open("/content/logo.jpg", "rb") as file:  # Update the path to your logo
        image = file.read()
    logo = widgets.Image(value=image, format='jpg', layout=widgets.Layout(height='100px', width='auto'))

    # Organize the layout
    top_buttons = widgets.HBox([admin_screen_button, statistics_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))
    vbox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
    vbox = widgets.VBox([top_buttons, logo, search_input, search_button, results_output], layout=vbox_layout)
    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh')
    container = widgets.Box(children=[vbox], layout=container_layout)

    # Display the elements
    display(container)

create_search_screen()


Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

In [386]:
def create_admin_screen():
    # Input text box
    input_text = widgets.Text(
        placeholder='Enter text here...',
        description='Query:',
        disabled=False,
        layout=widgets.Layout(width='100%', margin='0 0 10px 0'),
        style={'description_width': 'initial', 'border': '1px solid black'}
    )

    # Larger text area for displaying results
    results_text = widgets.Textarea(
        placeholder='Results will be shown here...',
        description='Results:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='200px', margin='0 0 10px 0'),
        style={'description_width': 'initial', 'border': '1px solid black'}
    )

    # Buttons for database operations
    delete_button = widgets.Button(description="Delete", button_style='danger', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    edit_button = widgets.Button(description="Edit", disabled=True, layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    add_button = widgets.Button(description="Add", button_style='success', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    get_button = widgets.Button(description="Get", button_style='info', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))

    # Actions for buttons
    def on_delete_button_clicked(b):
        # Code to handle delete action
        pass

    def on_edit_button_clicked(b):
        # Code to handle edit action
        pass

    def on_add_button_clicked(b):
        # Code to handle add action
        pass

    def on_get_button_clicked(b):
        edit_button.disabled = False
        results_text.value = "Fetched data from the database..."

    # Attach actions to buttons
    delete_button.on_click(on_delete_button_clicked)
    edit_button.on_click(on_edit_button_clicked)
    add_button.on_click(on_add_button_clicked)
    get_button.on_click(on_get_button_clicked)

    # Arrange buttons horizontally
    buttons_hbox = widgets.HBox([get_button, add_button, edit_button, delete_button], layout=widgets.Layout(justify_content='flex-start'))

    # Logo
    with open("/content/logo.jpg", "rb") as file:  # Update the path to your logo
        image = file.read()
    logo = widgets.Image(value=image, format='jpg', layout=widgets.Layout(height='100px', width='auto'))

    # Button to return to the previous page
    return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px'))

    # Function to handle return button click
    def on_return_button_clicked(b):
        container.children = []
        container.layout = widgets.Layout()
        create_search_screen()

    return_button.on_click(on_return_button_clicked)

    # Arrange all widgets vertically
    additional_string = widgets.Label("It's just seems to be easy")
    a_layout = widgets.Layout(display='flex', justify_content='space-between', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
    a = widgets.HBox(children=[return_button,additional_string ,logo], layout=a_layout)

    b_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
    b = widgets.VBox([input_text, results_text, buttons_hbox], layout=b_layout)

    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', justify_content='flex-start', height='100vh')
    container = widgets.Box(children=[a, b], layout=container_layout)

    # Display everything
    display(container)
